# Manipulating models

## Preparation

In [1]:
from cobra.io import read_sbml_model
model = read_sbml_model('data/e_coli_core.xml.gz')

## Making temporary changes to the model

Usually one relies on making copies if objects need to be changed but the original state needs to be retained and that's we what we did previously with the `model_copy = model.copy()`. Unfortunately, metabolic models can be enormous and copying those in memory can take quite long.

In [2]:
%%time
copy_of_model = model.copy()

CPU times: user 27.9 ms, sys: 6.15 ms, total: 34.1 ms
Wall time: 56.7 ms


Yes, even milliseconds add up pretty quickly if you need to run many simulation (e.g. if you need to knock out every single gene individually in the model to check if it is essential or not).

In [3]:
%%time
## avoid writing code like this...
for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

CPU times: user 3 s, sys: 204 ms, total: 3.2 s
Wall time: 4.38 s


For that reason cobrapy provides a mechanism that is less time consuming. Almost all methods that make changes to the mdoel such as knocking-out genes, reactions, adding or removing metabolites, reactions etc can be automatically reverted upon exit from a python context. How this works is probably best understood by looking at an example.

In [4]:
%%time
## instead do this; no copying!
with model:
    for gene in model.genes:
        gene.knock_out()

CPU times: user 41 ms, sys: 2.6 ms, total: 43.6 ms
Wall time: 46.9 ms


Here, the `with model` statements starts the context and changes done to the model one indentation level to the right, are automatically recorded. When that block finishes, the context manager is requested to roll-back all changes leaving the model looking exactly as it did before all the changes.

Changing flux bounds can as indicated also be done reversibly. For example let's set the lower and upper bound of phosphoglycerate kinase to 0 (effectively knocking out the reaction).

In [5]:
with model:
    model.reactions.PGK.bounds = 0, 0
    print("PGK's bounds inside the with statement")
    print(model.reactions.PGK.lower_bound, model.reactions.PGK.bounds)
    print('Mutant growth rate: ', model.optimize().objective_value)
print("PGK's bounds outside the with statement")
print(model.reactions.PGK.bounds)

PGK's bounds inside the with statement
0 (0, 0)
Mutant growth rate:  0.0
PGK's bounds outside the with statement
(-1000.0, 1000.0)


## Changing the medium

One can access the medium condition using `model.medium`. The indicated bound is the effective upper uptake bound. 

In [6]:
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

Changing the carbon source in the medium can be achieved by adjusting the flux bounds of the respective exchange reactions appropriately. For example, the following code block removes glucose from the medium and adds succinate.

In [7]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 0
    medium['EX_succ_e'] = 10
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])

0.397563015428


Changing the carbon source to succinate led to a significant drop in growth rate.

## Exercise (10 min)

* Change the carbon source in the medium to a different carbon source. What is the difference in the growth rate observed?
* How about growing E. coli under anaerobic conditions?

## Adding metabolites, reactions and pathways

In [8]:
from cobra import Reaction, Metabolite

Ok, let's create a new reactions.

In [9]:
new_reaction = Reaction('alchemy')

This reaction is going to convert water into gold (unfortunately lead is not part of _E. coli _ metabolism; creating wine would be blasphemy). So we need to create a new metabolite, since gold is not yet part of _E. coli's_ native metabolism.

In [10]:
gold = Metabolite(id='gold_c', compartment='c')

Now, we're going to specify the reaction's stoichiometry.

In [11]:
new_reaction.add_metabolites({model.metabolites.h2o_c: -1, gold: 1})

Printing the reaction reveals that the reaction indeed converts water into gold.

In [12]:
print(new_reaction.build_reaction_string())

h2o_c --> gold_c


Now, let's add the new reaction to the model.

In [13]:
model.add_reactions([new_reaction])

Quickly check that the reaction was indeed added to the model.

In [14]:
model.reactions.alchemy

Reaction identifier,alchemy
Name,
Memory address,0x0114772a20
Stoichiometry,h2o_c --> gold_c H2O -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Let's produce some gold then!

In [15]:
model.objective = model.reactions.alchemy
model.optimize().objective_value

0.0

:-(

What happened? Forgot to add an exchange reaction so that gold can leave the system.

In [16]:
model.add_boundary(model.metabolites.gold_c, type='demand')

Reaction identifier,DM_gold_c
Name,demand
Memory address,0x0114755a58
Stoichiometry,gold_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [17]:
model.objective = model.reactions.alchemy
model.optimize().objective_value

1000.0

Yes, much better!

## Exercise (20 min)

* Add a pathway to the model (ideally one that you're personally interested in; you can also use a different model if you like). 